In [1]:
from math import log10
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from dataset.super_resolution_data_loader import *
from SUB_CNN_coord_conv import Net
from pytorch_ssim import *

torch.manual_seed(1)
device = torch.device("cuda")

In [2]:
# SubPixelCNN CoordConv parameters

batch_size = 32
epochs = 40
lr = 0.01
threads = 4
upscale_factor = 4

In [3]:
#img_path_low = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-crap-56/train'
#img_path_ref = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-224/train'

img_path_low = '/media/angelo/DATEN/Datasets/CelebA/LR_56/test/'
img_path_ref = '/media/angelo/DATEN/Datasets/CelebA/HR/test/'

#img_path_low = '../dataset/300W-3D-crap-56/train'
#img_path_ref = '../dataset/300W-3D-low-res-224/train'

train_set = DatasetSuperRes(img_path_low, img_path_ref)
training_data_loader = DataLoader(dataset=train_set, num_workers=threads, batch_size=batch_size, shuffle=True)

In [4]:
print('===> Building model')
model = Net(upscale_factor=upscale_factor).to(device)
criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.2)

===> Building model


In [5]:
out_path = 'results/'
out_model_path = 'models/'

if not os.path.exists(out_path):
    os.makedirs(out_path)    

if not os.path.exists(out_model_path):
    os.makedirs(out_model_path)   
    
results = {'avg_loss': [], 'psnr': [], 'ssim': []}

In [7]:
def train(epoch):
    epoch_loss = 0
    model.train()
    for iteration, batch in enumerate(training_data_loader, 1):
        input_, target = batch[0].to(device), batch[1].to(device)
        
        optimizer.zero_grad()
        upsampled_img = model(input_)
        loss = criterion(upsampled_img, target)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()

        #print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(training_data_loader), loss.item()))
    
    scheduler.step() # Decrease learning rate after 100 epochs to 10% of its value
    
    psnr_epoch = 10*log10(1/(epoch_loss / len(training_data_loader)))
    ssim_epoch = ssim(upsampled_img, target).item()
    avg_loss_batch = epoch_loss/len(training_data_loader)
    
    results['psnr'].append(psnr_epoch)
    results['ssim'].append(ssim_epoch)
    results['avg_loss'].append(avg_loss_batch)
    
    print("===> Epoch {} Complete: Avg. Loss: {:.4f} / PSNR: {:.4f} / SSIM {:.4f}".format(epoch, 
                                                                                          avg_loss_batch, 
                                                                                          psnr_epoch,
                                                                                          ssim_epoch))
    if epoch % (epochs // 2) == 0:
    
        data_frame = pd.DataFrame(
                data={'Avg. Loss': results['avg_loss'], 'PSNR': results['psnr'], 'SSIM': results['ssim']},
                index=range(1, epoch + 1))

        data_frame.to_csv(out_path + 'SubCNN_coor_dx' + str(upscale_factor) + '_train_results.csv', index_label='Epoch')
        
        checkpoint(epoch)

def checkpoint(epoch):
    path = out_model_path + "SubCNN_coord_x{}_epoch_{}.pth".format(upscale_factor, epoch)
    torch.save(model, path)
    print("Checkpoint saved to {}".format(path))

In [8]:
for epoch in range(1, epochs + 1):
    train(epoch)

===> Epoch 1 Complete: Avg. Loss: 0.1603 / PSNR: 7.9496 / SSIM 0.5121
===> Epoch 2 Complete: Avg. Loss: 0.0081 / PSNR: 20.9015 / SSIM 0.6369
===> Epoch 3 Complete: Avg. Loss: 0.0049 / PSNR: 23.1270 / SSIM 0.6762
===> Epoch 4 Complete: Avg. Loss: 0.0040 / PSNR: 24.0202 / SSIM 0.7329
===> Epoch 5 Complete: Avg. Loss: 0.0034 / PSNR: 24.6359 / SSIM 0.7764
===> Epoch 6 Complete: Avg. Loss: 0.0027 / PSNR: 25.6132 / SSIM 0.7840
===> Epoch 7 Complete: Avg. Loss: 0.0025 / PSNR: 25.9942 / SSIM 0.7810
===> Epoch 8 Complete: Avg. Loss: 0.0023 / PSNR: 26.3666 / SSIM 0.7905
===> Epoch 9 Complete: Avg. Loss: 0.0022 / PSNR: 26.5097 / SSIM 0.8268
===> Epoch 10 Complete: Avg. Loss: 0.0019 / PSNR: 27.1667 / SSIM 0.8163
===> Epoch 11 Complete: Avg. Loss: 0.0018 / PSNR: 27.5687 / SSIM 0.8272
===> Epoch 12 Complete: Avg. Loss: 0.0017 / PSNR: 27.6556 / SSIM 0.8248
===> Epoch 13 Complete: Avg. Loss: 0.0017 / PSNR: 27.7012 / SSIM 0.8053
===> Epoch 14 Complete: Avg. Loss: 0.0017 / PSNR: 27.7569 / SSIM 0.7778
==